In [2]:
import pandas as pd
import numpy as np

### 練習一：

一生物實驗會有三種結果，經驗上三類的比例應分別為$(1-\theta)^2, 2\theta(1-\theta),$ 與 $\theta^2$。今觀察到三類的次數分布為 343，501，186。判斷蒐集到的資料是否與經驗相衝突？ (設顯著水準 0.05) 

In [3]:
sample=[343, 501, 186]
N=sum(sample)

In [4]:
from scipy.optimize import minimize

def ss(theta):    
    n1=N*(1-theta)**2
    n2=N*2*theta*(1-theta)
    n3=N*theta**2
    return((n1-sample[0])**2+(n2-sample[1])**2+(n3-sample[2])**2)

initParams = [0.5]
theta_est = minimize(ss, initParams,  bounds=[(0,1)],method='L-BFGS-B').x

print('參數估計 theta_hat=',theta_est)


參數估計 theta_hat= [0.4233621]


In [5]:
from scipy.stats import chi2

theta=theta_est[0]
Ei=[N*(1-theta)**2,N*2*theta*(1-theta),N*theta**2]
X=np.sum(np.square(np.array(sample)-np.array(Ei))/np.array(Ei))

df=3-1-1
X_alpha=chi2.isf(0.05, df)
    
if X>=X_alpha:
    print("reject H0 where H0: data follow past experience")
else:
    print("do not reject H0 where H0: data follow past experience")

do not reject H0 where H0: data follow past experience


### 練習二：

In [6]:
data1=pd.DataFrame(data=[[68, 119, 26, 7],[20, 84, 17, 94],[15, 54, 14, 10],[5, 29, 14, 16]], 
                   columns=['hair_black','hair_brown','hair_red','hair_bond'], 
                   index=['eye_brown','eye_blue','eye_hazel','eye_green'])
data1

,hair_black,hair_brown,hair_red,hair_bond
eye_brown,68,119,26,7
eye_blue,20,84,17,94
eye_hazel,15,54,14,10
eye_green,5,29,14,16


In [7]:
from scipy.stats import chi2

def chi_square_text(data, alpha):
    R_n=data.shape[0]
    C_n=data.shape[1]
    Ri=sum(data.T.values)
    Cj=sum(data.values)
    N=sum(Ri)
    Eij=pd.DataFrame(np.repeat(np.nan, R_n*C_n).reshape(R_n, C_n))
    for i in range(R_n):
        for j in range(C_n):
            Eij.loc[i,j]=(Ri[i]*Cj[j])/N
    
    Oij=np.matrix(data)
    Eij=np.matrix(Eij)
    X=np.sum( np.square(Oij-Eij)/Eij )
    
    df=(R_n-1)*(C_n-1)
    X_alpha=chi2.isf(alpha, df)
    
    V=np.sqrt(X/(N*min(R_n-1, C_n-1)))
    
    if X>=X_alpha:
        print("reject H0 where H0: two variable are independent, and Cramer's V=", V)
    else:
        print("do not reject H0 where H0: two variable are independent, and Cramer's V=", V)


In [8]:
chi_square_text(data1, alpha=0.05)

reject H0 where H0: two variable are independent, and Cramer's V= 0.2790446233426584


### 練習三：

In [11]:
data=pd.DataFrame({'a':[18,21,19,18,16],'b':[20,19,20,22,21],'c':[23,21,18,25,24]})
data

,a,b,c
0,18,20,23
1,21,19,21
2,19,20,18
3,18,22,25
4,16,21,24


In [32]:
def R_square(data):
    n=data.shape[0] #資料中每組個數
    m=data.shape[1] #資料組數
    SSE=np.sum(np.var(data)*n)
    
    tmp=np.array(data).reshape(n*m,1)
    all_mean=np.mean(tmp)
    SST=np.sum((tmp-all_mean)**2)
    
    score=1-(SSE/SST)
    return(score)

In [18]:
print('排序前資料的R^2=', R_square(data))

排序前資料的R^2= 0.4234375


#### 用排序後資料分析

In [44]:
df=pd.DataFrame(data.values.flatten(),columns=['value'])
df['rank']=df['value'].rank()
rdata=pd.DataFrame({'a':df['rank'].values[0:5],'b':df['rank'].values[5:10],'c':df['rank'].values[10:15]})
rdata

,a,b,c
0,3.0,10.0,12.0
1,7.5,5.5,15.0
2,13.0,7.5,1.0
3,10.0,3.0,10.0
4,5.5,3.0,14.0


In [45]:
df['rank']

0      3.0
1      7.5
2     13.0
3     10.0
4      5.5
5     10.0
6      5.5
7      7.5
8      3.0
9      3.0
10    12.0
11    15.0
12     1.0
13    10.0
14    14.0
Name: rank, dtype: float64

In [17]:
print('排序後資料的R^2=', R_square(rdata))

排序後資料的R^2= 0.19345454545454555


### 練習五：

In [17]:
from scipy.stats import pearsonr, spearmanr

data=pd.read_csv('data.csv')

dataset=data.dataset.unique()
x_mean=[]
x_var=[]
y_mean=[]
y_var=[]
xy_pearson_corr=[]

for i in dataset:
    tmp=data[data.dataset==i]
    xmean=np.mean(tmp.x)
    x_mean.append(xmean)
    xvar=np.var(tmp.x, ddof=1)
    x_var.append(xvar)
    ymean=np.mean(tmp.y)
    y_mean.append(ymean)
    yvar=np.var(tmp.y, ddof=1)
    y_var.append(yvar)
    pear_corr, pear_p_value =pearsonr(tmp.x, tmp.y)
    xy_pearson_corr.append(pear_corr)
    
    
    

In [18]:
df=pd.DataFrame({'dataset':dataset, 
                 'x_mean':x_mean, 'x_var':x_var,
                'y_mean':y_mean, 'y_var':y_var,
                'pearson_corr':xy_pearson_corr})

df

,dataset,x_mean,x_var,y_mean,y_var,pearson_corr
0,dino,54.263273,281.069988,47.832253,725.515961,-0.064472
1,away,54.266100,281.227029,47.834721,725.749775,-0.064128
2,h_lines,54.261442,281.095333,47.830252,725.756931,-0.061715
3,v_lines,54.269927,281.231512,47.836988,725.638809,-0.069446
4,x_shape,54.260150,281.231481,47.839717,725.224991,-0.065583
5,star,54.267341,281.197993,47.839545,725.239695,-0.062961
6,high_lines,54.268805,281.122364,47.835450,725.763490,-0.068504
7,dots,54.260303,281.156953,47.839829,725.235215,-0.060341
8,circle,54.267320,280.898024,47.837717,725.226844,-0.068343
9,bullseye,54.268730,281.207393,47.830823,725.533372,-0.068586
